<a href="https://colab.research.google.com/github/shreyasat27/pennylane-27524/blob/main/qubit_rotation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install pennylane

In [20]:
import pennylane as qml
from jax import numpy as np
import jax

create a device

In [21]:
dev1= qml.device("lightning.qubit", wires=1)

construct a QNode

In [22]:
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

we convert it into a QNode running on device dev1 by applying the qnode() decorator. directly above the function definition:

In [23]:
@qml.qnode(dev1)
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

quantum fucntion is now a QNode. now we will simply all the function with some appropriate numerical values

In [28]:
params = np.array([0.54, 0.12])


In [29]:
print(circuit(params))

0.85154057


**Calculating quantum gradients**

PennyLane incorporates both analytic differentiation, as well as numerical methods (such as the method of finite differences). Both of these are done automatically.

We can differentiate by using the jax.grad function. This returns another function, representing the gradient (i.e., the vector of partial derivatives) of circuit. The gradient can be evaluated in the same way as the original function:

In [25]:
dcircuit= jax.grad(circuit, argnums=0)
print(dcircuit(params))

[-0.5104387  -0.10267819]


In [26]:
@qml.qnode(dev1)
def circuit2(phi1, phi2):
    qml.RX(phi1, wires=0)
    qml.RY(phi2, wires=0)
    return qml.expval(qml.PauliZ(0))

In [27]:
phi1 = np.array([0.54])
phi2 = np.array([0.12])

dcircuit = jax.grad(circuit2, argnums=[0, 1])
print(dcircuit(phi1, phi2))

(Array([-0.5104387], dtype=float32), Array([-0.10267819], dtype=float32))


Optimization

In [32]:
def cost(x):
    return circuit(x)

In [33]:
init_params =np.array([0.011, 0.012])
print(cost(init_params))

0.9998675


We can see that, for these initial parameter values, the cost function is close to  
1
 .

Finally, we use an optimizer to update the circuit parameters for 100 steps. We can use the gradient descent optimizer:



In [35]:
!pip install jaxopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 5.4 MB/s eta 0:00:00


In [41]:
import jaxopt

#initialise the optimizer
opt = jaxopt.GradientDescent(cost, stepsize=0.4, acceleration = False)

#set the number of steps
steps = 40

#se the initial parameter values
params = init_params
opt_state = opt.init_state(params)

for i in range(steps):
  #update the circuit parameters
    params, opt_state = opt.update(params, opt_state)

    if (i+1) % 5 == 0:
      print("cost after step {:5f}: {: .7f}".format(i+1, cost(params)))
print("optimized totation angels: {}".format(params))

cost after step 5.000000:  0.9961779
cost after step 10.000000:  0.8974943
cost after step 15.000000:  0.1440490
cost after step 20.000000: -0.1536721
cost after step 25.000000: -0.9152496
cost after step 30.000000: -0.9994046
cost after step 35.000000: -0.9999964
cost after step 40.000000: -1.0000000
optimized totation angels: [1.4634992e-04 3.1414437e+00]
